In [3]:
from transform import preprocessing

teams_division_2 = ['수원', '전남', '천안', '안산', '서울E', '성남', '김포', '충북청주', '안양', '충남아산', '부산', '부천']

df = preprocessing(27)
# df = df[~df['구단'].isin(teams_division_2)]

Data has been written to data/preprocessed/27-round-preprocessed.csv


In [4]:
print(df['구단'].value_counts())

구단
김천      47
대전      47
전북      45
수원      43
천안      40
전남      39
강원      39
수원FC    39
제주      39
울산      39
서울      38
경남      38
광주      38
안산      37
서울E     37
대구      36
포항      36
성남      35
김포      35
충북청주    35
안양      35
충남아산    34
부산      34
인천      34
부천      32
Name: count, dtype: int64


In [5]:
for column in df.columns:
    print(column)

index
선수명
구단
포지션
등번호
출전시간(분)
득점
도움
슈팅
유효 슈팅
차단된슈팅
벗어난슈팅
PA내 슈팅
PA외 슈팅
오프사이드
프리킥
코너킥
스로인
드리블 시도
드리블 성공
드리블 성공%
패스 시도
패스 성공
패스 성공%
키패스
전방 패스 시도
전방 패스 성공
전방 패스 성공%
후방 패스 시도
후방 패스 성공
후방 패스 성공%
횡패스 시도
횡패스 성공
횡패스 성공%
공격지역패스 시도
공격지역패스 성공
공격지역패스 성공%
수비지역패스 시도
수비지역패스 성공
수비지역패스 성공%
중앙지역패스 시도
중앙지역패스 성공
중앙지역패스 성공%
롱패스 시도
롱패스 성공
롱패스 성공%
중거리패스 시도
중거리패스 성공
중거리패스 성공%
숏패스 시도
숏패스 성공
숏패스 성공%
크로스 시도
크로스 성공
크로스 성공%
경합 지상 시도
경합 지상 성공
경합 지상 성공%
경합 공중 시도
경합 공중 성공
경합 공중 성공%
태클 시도
태클 성공
태클 성공%
클리어링
인터셉트
차단
획득
블락
볼미스
파울
피파울
경고
퇴장
xG
득점/xG
Defensive Action
Possession won
득점/90
도움/90
슈팅/90
유효 슈팅/90
차단된슈팅/90
벗어난슈팅/90
PA내 슈팅/90
PA외 슈팅/90
오프사이드/90
프리킥/90
코너킥/90
스로인/90
드리블 시도/90
드리블 성공/90
패스 시도/90
패스 성공/90
키패스/90
전방 패스 시도/90
전방 패스 성공/90
후방 패스 시도/90
후방 패스 성공/90
횡패스 시도/90
횡패스 성공/90
공격지역패스 시도/90
공격지역패스 성공/90
수비지역패스 시도/90
수비지역패스 성공/90
중앙지역패스 시도/90
중앙지역패스 성공/90
롱패스 시도/90
롱패스 성공/90
중거리패스 시도/90
중거리패스 성공/90
숏패스 시도/90
숏패스 성공/90
크로스 시도/90
크로스 성공/90
경합 지상 시도/90
경합 지상 성공/90
경합 공중 시도/90
경합 공중 성공/90
태클 시도/90
태클 성공/90
클리어링/90
인터셉트/9

In [13]:
import numpy as np
from scipy.stats import rankdata
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

class RadarChart(object):
    def __init__(self, player_name, team_name, round_number, df):
        self.fig = go.Figure()
        self.df = df
        self.player_name = player_name
        self.team_name = team_name
        self.round_number = round_number
        self.player_position = self.df[self.df['선수명'] == player_name]["포지션"].values[0]
        self.string_column = ['index', '선수명', '구단', '포지션', '등번호']
        self.radar_columns = ['득점/xG', '슈팅/90', '유효 슈팅/90', '드리블 시도/90', '공격지역패스 성공%', '키패스/90', 'Defensive Action/90', 'Possession won/90', '경합 공중 성공/90']

        # 특정 선수의 포지션과 팀 기준으로 필터링
        self.relative_df = df[df['포지션'] == self.player_position]
        self.relative_df = self.relative_df[self.relative_df['출전시간(분)'] >= 450]
        
        # 퍼센타일 스케일링
        def percentile_rank(x):
            # 모든 값이 동일한 경우를 처리
            if len(np.unique(x)) == 1:
                return np.zeros(len(x))
            ranks = rankdata(x)
            scaled = 100 * (ranks - 1) / (len(ranks) - 1)
            return scaled
        
        self.radar_scaled_df = self.relative_df.copy()
        self.radar_scaled_df[self.radar_columns] = self.relative_df[self.radar_columns].apply(percentile_rank)
        
        # 특정 선수의 데이터 추출
        self.target_df = self.radar_scaled_df[self.radar_scaled_df["선수명"] == player_name][self.radar_columns]
        self.key_value = '키패스/90'
        print(self.radar_scaled_df.sort_values(by=self.key_value, ascending=False).head(10))

    def plot(self):
        if self.target_df.empty:
            print(f"No data available for player {self.player_name}")
            return

        categories = self.radar_columns
        values = self.target_df.values.flatten().tolist()

        # Close the loop
        values += values[:1]
        categories += categories[:1]

        self.fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            name=self.player_name
        ))

        self.fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    range=[0, 100],
                ),
                angularaxis=dict(
                    showgrid=False,
                )
            ),
            plot_bgcolor='rgb(0,22,72)',
            paper_bgcolor='rgb(0,89,167)',
            title=f"{self.round_number} Round - {self.player_name} Radar Chart"
        )

        self.fig.show()
        self.fig.write_image(f"output/{self.round_number}round-{self.player_name}-radar.png", format='png')

    def save_as_html(self, filepath="output"):
        if not filepath.endswith('.html'):
            filepath = f"{filepath}/{self.round_number}round-{self.player_name}-radar.html"
        
        # Save HTML with external plotly.js
        pio.write_html(self.fig, file=filepath, auto_open=False, include_plotlyjs='include')
        print(f"HTML file saved to {filepath}")


In [22]:
mh_radar = RadarChart('김진호', '광주', 27, df)
mh_radar.plot()
mh_radar.save_as_html()

     index  선수명   구단 포지션   등번호  출전시간(분)   득점   도움    슈팅  유효 슈팅  ...     획득/90  \
287  287.0  박대원   김천  DF  48.0    452.0  0.0  1.0   3.0    0.0  ...  5.973451   
846  843.0  최우진   인천  DF  14.0   1331.0  0.0  4.0  11.0    2.0  ...  6.220887   
134  134.0  김영욱   안산  DF  14.0    645.0  0.0  0.0   3.0    1.0  ...  8.232558   
133  133.0  김영욱  서울E  DF  14.0    645.0  0.0  0.0   3.0    1.0  ...  8.232558   
827  824.0  최강민   울산  DF  70.0    462.0  1.0  0.0   7.0    4.0  ...  7.402597   
566  565.0  이기제   수원  DF  23.0    947.0  0.0  0.0   4.0    2.0  ...  6.272439   
236  236.0  두현석   광주  DF  13.0   1449.0  0.0  3.0   7.0    2.0  ...  7.701863   
713  711.0  장시영   울산  DF  28.0    567.0  1.0  0.0   3.0    1.0  ...  5.396825   
188  188.0  김진수   전북  DF  23.0   1813.0  0.0  3.0  10.0    1.0  ...  6.503034   
427  426.0  신재원   성남  DF  17.0   1024.0  0.0  4.0  10.0    1.0  ...  4.042969   

        블락/90    볼미스/90     파울/90    피파울/90     경고/90     퇴장/90  \
287  1.792035  0.398230  1.792035  0.3982

HTML file saved to output/27round-김진호-radar.html


In [69]:
import plotly.express as px

class ScatterChart(object):
    def __init__(self, x, y, player_name, df):
        self.player_name = player_name
        # self.player_position = df[df['선수명'] == player_name]['포지션'].values[0]
        # self.df = df[(df['출전시간(분)'] >= 1000) & (df['포지션'] == self.player_position)]
        self.df = df[df['출전시간(분)'] >= 1000]
        self.x = x
        self.y = y
        
        # Calculate the mean values for x and y axes
        self.mean_x = self.df[self.x].mean()
        self.mean_y = self.df[self.y].mean()

        # Create the scatter plot
        self.fig = px.scatter(
            self.df,
            x=self.x,
            y=self.y,
            hover_name='선수명', # Set player names to appear on hover
            color_discrete_sequence=['lightgray'],  # Set the default color to lightgray
        )
        
        # Add mean lines
        self.add_mean_lines()

    def add_mean_lines(self):
        # Add vertical and horizontal lines for the mean values
        self.fig.add_shape(
            type="line",
            x0=self.mean_x,
            y0=self.df[self.y].min(),
            x1=self.mean_x,
            y1=self.df[self.y].max(),
            line=dict(color="gray", width=1),
        )
        self.fig.add_shape(
            type="line",
            x0=self.df[self.x].min(),
            y0=self.mean_y,
            x1=self.df[self.x].max(),
            y1=self.mean_y,
            line=dict(color="gray", width=1),
        )

    def plot(self):
        self.fig.update_layout(
            xaxis=dict(showgrid=False),  # Hide x-axis grid
            yaxis=dict(showgrid=False),  # Hide y-axis grid
        )
        # Replace invalid characters in filename
        safe_x = self.x.replace("/", "_")
        safe_y = self.y.replace("/", "_")
        filename = f'output/{self.player_name}_scatter_{safe_x}_{safe_y}.png'
        self.fig.write_image(filename, format='png')
        self.fig.show()


In [70]:
for column in df.columns:
    print(column)

index
선수명
구단
포지션
등번호
출전시간(분)
득점
도움
슈팅
유효 슈팅
차단된슈팅
벗어난슈팅
PA내 슈팅
PA외 슈팅
오프사이드
프리킥
코너킥
스로인
드리블 시도
드리블 성공
드리블 성공%
패스 시도
패스 성공
패스 성공%
키패스
전방 패스 시도
전방 패스 성공
전방 패스 성공%
후방 패스 시도
후방 패스 성공
후방 패스 성공%
횡패스 시도
횡패스 성공
횡패스 성공%
공격지역패스 시도
공격지역패스 성공
공격지역패스 성공%
수비지역패스 시도
수비지역패스 성공
수비지역패스 성공%
중앙지역패스 시도
중앙지역패스 성공
중앙지역패스 성공%
롱패스 시도
롱패스 성공
롱패스 성공%
중거리패스 시도
중거리패스 성공
중거리패스 성공%
숏패스 시도
숏패스 성공
숏패스 성공%
크로스 시도
크로스 성공
크로스 성공%
경합 지상 시도
경합 지상 성공
경합 지상 성공%
경합 공중 시도
경합 공중 성공
경합 공중 성공%
태클 시도
태클 성공
태클 성공%
클리어링
인터셉트
차단
획득
블락
볼미스
파울
피파울
경고
퇴장
xG
득점/xG
Defensive Action
Possession won
득점/90
도움/90
슈팅/90
유효 슈팅/90
차단된슈팅/90
벗어난슈팅/90
PA내 슈팅/90
PA외 슈팅/90
오프사이드/90
프리킥/90
코너킥/90
스로인/90
드리블 시도/90
드리블 성공/90
패스 시도/90
패스 성공/90
키패스/90
전방 패스 시도/90
전방 패스 성공/90
후방 패스 시도/90
후방 패스 성공/90
횡패스 시도/90
횡패스 성공/90
공격지역패스 시도/90
공격지역패스 성공/90
수비지역패스 시도/90
수비지역패스 성공/90
중앙지역패스 시도/90
중앙지역패스 성공/90
롱패스 시도/90
롱패스 성공/90
중거리패스 시도/90
중거리패스 성공/90
숏패스 시도/90
숏패스 성공/90
크로스 시도/90
크로스 성공/90
경합 지상 시도/90
경합 지상 성공/90
경합 공중 시도/90
경합 공중 성공/90
태클 시도/90
태클 성공/90
클리어링/90
인터셉트/9

In [72]:
scatter = ScatterChart('xG', '득점', '이승우', df)
scatter.plot()

In [23]:
print(df.shape)

(836, 134)
